In [3]:
import torch
from ResNet import resnet18, resnet50

from imagenet_dataloader import ImageNetwithLUAB_dataloader
from imagenet_extended_dataloader import ImageNetwithLUAB_extended_dataloader


/scratch_local/owl156-448013/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
 = "/mnt/lustre/datasets/ImageNet2012/train"
xml_path = "/scratch_local/owl156-448013/train_xml"
nr_epochs = 300
batch_size = 64
input_size = 224
num_workers = 8
num_class = 1000

loader = ImageNetwithLUAB_extended_dataloader(
    root_train=root_train,
    xml_path=xml_path,
    num_classes=num_class,
    input_size=input_size,
    batch_size=batch_size,
    num_workers=num_workers,
    loss_weight=1,
).run()

In [6]:
a = iter(loader)

In [8]:
image, (target, weight, fg_point, loc_info, mouse_record) = next(a)

In [22]:
i = -1
mouse_record[i]

(tensor(1., dtype=torch.float64),
 tensor([[ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1],
         [-1, -1]]))

In [ ]:

(inputs, ab_information) = next(iter(loader))
print('Dataloader sanity check, this should be (batch_size, (2))', ab_information[-1].shape)
model = resnet50().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
if args.compile:
    model = torch.compile(model, mode="reduce-overhead")

L_image = torch.nn.CrossEntropyLoss()
L_ab = torch.nn.SmoothL1Loss()
print('---Training---')
for epoch in range(nr_epochs):
    total_loss = 0
    for batch_idx, (inputs, target) in enumerate(loader):
        targets, weight, fg_point, loc_info = ab_information
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        visual_outputs, ab_outputs = model(inputs)
        loss = L_image(visual_outputs, targets)
        if loc_info is not None:
            loc_info = loc_info.cuda()
            loss += args.lambda_ * L_ab(ab_outputs, loc_info)

        loss.backward()
        optimizer.step()
        total_loss+=loss
    print(f"[Training] - E{epoch} - Loss {total_loss}")
